# 手写双层的神经网络进行波士顿房价预测

 Paddle训练营有讲过单层的写法，这里写一下双层的
 
 
### 构建神经网络


In [14]:
import numpy as np

class MultiNetwork(object):
    
    def __init__(self, num_of_weights):
        # 随机初始化 w
        # 为保证每次运行结果一致，这里设置了固定随机数种子
        np.random.seed(0)
        
        #这里有一层输入神经元，一层隐层分别定义权重为
        self.num_of_weights = num_of_weights
        self.v = np.random.randn(num_of_weights * num_of_weights,1).reshape(num_of_weights,num_of_weights)
        self.w = np.random.randn(num_of_weights * 1 ,1)
        
        # 对应每层阈值
        self.gamma = np.random.randn(num_of_weights,1) # γ
        self.thet = 0.      # θ
        
   
    def forward(self,x):
        # 定义两层的前项计算
        
        # 隐层结果 bh
        
        # 注：这种写法不能使用 numpy 的批量处理
        b_h = np.zeros(13)
        for i in range(self.num_of_weights):
            temp =  np.dot(x, self.v[i]) + self.gamma[i]
            b_h[i] = temp
        
        # or
        # b_h = np.dot(x,self.v) + self.gamma
        
        # print("隐层结果",b_h)
        
        # 这里隐层结果 b_h 可以直接传给下一层，也可以使用 Sigmoid处理一下。
        # 目前直接传到下层做下层输入，如下：
        
        # 输出层神经元，预测结果
        _yk = np.dot(b_h,self.w) + self.thet
        
        return _yk, b_h
        
        
    def loss(self, z, y):
        # z,y 可以是向量
        error = z-y
        cost = error * error
        cost = np.mean(cost)
        return cost

    def gradient(self, x , y ):
        
        _yk, b_h = self.forward(x)
        
        # g是中间变量，根据公式推导的简化变量
        g = y-_yk
        
        gradient_w = g * b_h
        #查格式化对齐
        
        gradient_thet = g
        
        
        # 注：这种写法不能使用 numpy 广播
        gradient_v = np.zeros([13,13])
        for i in range(self.w.size):
            gradient_v[i] = g * self.w[i] * x
            
        # gradient_v = g * self.w * x # 这个地方可能numpy 直接这样算的不对
        
        gradient_gamma = g * self.w
        
        return gradient_w, gradient_thet,gradient_v,gradient_gamma
    






multi_network = MultiNetwork(13)

# print("第一层权重 v: ",multi_network.v,"\n 阈值 gamma：",multi_network.gamma)
# print("第二层权重 w: ",multi_network.w,"\n 阈值 thet：",multi_network.thet)
 

In [ ]:
# 测试的代码

# test_x = np.random.randn(13)
# print("模拟测试数据test_x为: ",test_x)
# y = [1]
# z,b_n = multi_network.forward(test_x)
# print('predict: ', z)
# loss = multi_network.loss(z,y)
# print(loss)
# 
# # 最好再画个图
# 
# 
# # 计算梯度
# gradients =  multi_network.gradient(test_x,y)
# 
# print ("w层梯度",gradients[0])
# print ("θ梯度",gradients[1])
# 
# print ("v层梯度",gradients[2])
# print ("gamma梯度",gradients[3])




### 定义加载数据


In [15]:
def load_data():
    # 从文件导入数据
    datafile = 'resources/housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]

    # 对数据进行归一化处理
    for i in range(feature_num):
        #print(maximums[i], minimums[i], avgs[i])
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data



### 获取数据
> 这里数据放在了 resource 下的 housing.data


In [16]:
training_data, test_data = load_data()
x = training_data[:,:-1]
y = training_data[:,-1:]
print(x[0])
print(y[0])



[-0.02146321  0.03767327 -0.28552309 -0.08663366  0.01289726  0.04634817
  0.00795597 -0.00765794 -0.25172191 -0.11881188 -0.29002528  0.0519112
 -0.17590923]
[-0.00390539]


### 每次使用1条数据测试下
>目前用 numpy 的广播机制没调试好


In [17]:
x1samples = x[0]
y1samples = y[0]
z1samples,b_h = multi_network.forward(x1samples)

print('x {}, shape {}'.format(x1samples, x1samples.shape))
print('y {}, shape {}'.format(y1samples, y1samples.shape))
print('z {}, shape {}'.format(z1samples, z1samples.shape))

gradients = multi_network.gradient(x1samples,y1samples)
print(gradients[0])

x [-0.02146321  0.03767327 -0.28552309 -0.08663366  0.01289726  0.04634817
  0.00795597 -0.00765794 -0.25172191 -0.11881188 -0.29002528  0.0519112
 -0.17590923], shape (13,)
y [-0.00390539], shape (1,)
z [-4.5899854], shape (1,)
[-5.84928579 -9.57440199 -0.35484424 -5.18668274 -3.79429499  2.2511544
 -1.75711525  4.89430493 -4.43775868 -2.15837976  3.08573478 -6.07094353
  5.1498    ]
